In [3]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 14.3 MB/s eta 0:00:00


In [ ]:
class KinesisStream:
    """Encapsulates a Kinesis stream."""

    def __init__(self, kinesis_client):
        """
        :param kinesis_client: A Boto3 Kinesis client.
        """
        self.kinesis_client = kinesis_client
        self.name = None
        self.details = None
        self.stream_exists_waiter = kinesis_client.get_waiter("stream_exists")


    def put_record(self, data, partition_key):
        """
        Puts data into the stream. The data is formatted as JSON before it is passed
        to the stream.

        :param data: The data to put in the stream.
        :param partition_key: The partition key to use for the data.
        :return: Metadata about the record, including its shard ID and sequence number.
        """
        try:
            response = self.kinesis_client.put_record(
                StreamName=self.name, Data=json.dumps(data), PartitionKey=partition_key
            )
            logger.info("Put record in stream %s.", self.name)
        except ClientError:
            logger.exception("Couldn't put record in stream %s.", self.name)
            raise
        else:
            return response

In [5]:
import configparser

sample_config = """

[default]
aws_access_key_id=
aws_secret_access_key=
aws_session_token=

"""
config = configparser.ConfigParser(allow_no_value=True)
config.read_string(sample_config)

aws_access_key_id = config["default"]["aws_access_key_id"]
aws_secret_access_key = config["default"]["aws_secret_access_key"]
aws_session_token = config["default"]["aws_session_token"]

In [6]:
import boto3
import json

session = boto3.Session(
   aws_access_key_id=aws_access_key_id,
   aws_secret_access_key=aws_secret_access_key,
   aws_session_token=aws_session_token
)

kinesis_client = session.client(
   "kinesis",
   region_name='us-east-1'
)

In [7]:
community_areas = {
  "1":  "Rogers Park",
  "2":  "West Ridge",
  "3":  "Uptown",
  "4":  "Lincoln Square",
  "5":  "North Center",
  "6":  "Lake View",
  "7":  "Lincoln Park",
  "8":  "Near North Side",
  "9":  "Edison Park ",
  "10":     "Norwood Park",
  "11":     "Jefferson Park",
  "12":     "Forest Glen",
  "13":     "North Park",
  "14":     "Albany Park",
  "15":     "Portage Park",
  "16":     "Irving Park",
  "17":     "Dunning",
  "18":     "Montclare",
  "19":     "Belmont Cragin",
  "20":     "Hermosa",
  "21":     "Avondale",
  "22":     "Logan Square",
  "23":     "Humboldt Park",
  "24":     "West Town",
  "25":     "Austin",
  "26":     "West Garfield Park",
  "27":     "East Garfield Park",
  "28":     "Near West Side",
  "29":     "North Lawndale",
  "30":     "South Lawndale",
  "31":     "Lower West Side",
  "32":     "Loop",
  "33":     "Near South Side",
  "34":     "Armour Square",
  "35":     "Douglas",
  "36":     "Oakland",
  "37":     "Fuller Park",
  "38":     "Grand Boulevard",
  "39":     "Kenwood",
  "40":     "Washington Park",
  "41":     "Hyde Park",
  "42":     "Woodlawn",
  "43":     "South Shore",
  "44":     "Chatham",
  "45":     "Avalon Park",
  "46":     "South Chicago",
  "47":     "Burnside",
  "48":     "Calumet Heights",
  "49":     "Roseland",
  "50":     "Pullman",
  "51":     "South Deering",
  "52":     "East Side",
  "53":     "West Pullman",
  "54":     "Riverdale",
  "55":     "Hegewisch",
  "56":     "Garfield Ridge",
  "57":     "Archer Heights",
  "58":     "Brighton Park",
  "59":     "McKinley Park",
  "60":     "Bridgeport",
  "61":     "New City",
  "62":     "West Elsdon",
  "63":     "Gage Park",
  "64":     "Clearing",
  "65":     "West Lawn",
  "66":     "Chicago Lawn",
  "67":     "West Englewood",
  "68":     "Englewood",
  "69":     "Greater Grand Crossing",
  "70":     "Ashburn",
  "71":     "Auburn Gresham",
  "72":     "Beverly" ,
  "73":     "Washington Heights",
  "74":     "Mount Greenwood",
  "75":     "Morgan Park",
  "76":     "O'Hare",
  "77":     "Edgewater"
}

In [8]:
import requests
from datetime import datetime

class BigCrimeDataReceiver:
  def __init__(self, app_token):
    self.app_token = app_token

  def getRangeDates(self):
    url = f"https://data.cityofchicago.org/resource/ijzp-q8t2.json?$$app_token={self.app_token}&$select=MIN(date),MAX(date)"
    response = requests.get(url)
    response_json = response.json()
    return (
      datetime.fromisoformat(response_json[0]["MIN_date"]).date(),
      datetime.fromisoformat(response_json[0]["MAX_date"]).date()
    )

  def getData(self, offset, limit, date_from, date_to):
    url = f"https://data.cityofchicago.org/resource/ijzp-q8t2.json?$$app_token={self.app_token}&$limit={limit}&$offset={offset}&$order=date&$where=date between '{date_from}' and '{date_to}'"
    response = requests.get(url)
    return response.json()





In [51]:
data_receiver = BigCrimeDataReceiver(
   app_token='yMG7ud7wf0svusOppJSWAJmPG'
)
data_receiver.getRangeDates()

(datetime.date(2001, 1, 1), datetime.date(2024, 5, 29))

In [18]:
step = 10
size = 100
for x in range(0, size-1, step):
  print(f'{x} - {x+step-1}')

0 - 9
10 - 19
20 - 29
30 - 39
40 - 49
50 - 59
60 - 69
70 - 79
80 - 89
90 - 99


In [23]:
arr = [1,2,3,4,5,6,7,8,9,0]
arr[6:20]

[7, 8, 9, 0]

In [30]:
import requests
import time
import random
from datetime import datetime, date, timedelta

data_receiver = BigCrimeDataReceiver(
   app_token='yMG7ud7wf0svusOppJSWAJmPG'
)
(min_date, max_date) = data_receiver.getRangeDates()

limit = 10000
offset = 0

# date_from = '2024-04-29T00:00:00'
# date_to = '2024-05-05T23:59:59'
date_from = min_date
count = 0
ts_1 = time.time()

while date_from < max_date:
  date_to = date_from + timedelta(days=7)
  print(f'requesting data from {date_from} to {date_to}')
  while True:
    response_json = data_receiver.getData(offset, limit, date_from, date_to)
    # print(response_json[0])
    size = len(response_json)
    # print(f'response_size={size}')
    step = 100
    for x in range(0, size - 1, step):
      # print(f'{x} - {x + step - 1}')
      part = response_json[x : x + step - 1]
      for_kinesis = []
      for record in part:
        # print(i, "\n")
        record['date'] = datetime.fromisoformat(record['date']).strftime("%Y-%m-%d %H:%M:%S")
        if 'community_area' in record : record['community_area'] = community_areas[record['community_area']]
        # print(i, "\n")
        for_kinesis.append({
          'Data': json.dumps(record),
          'PartitionKey': record['block']
        })

      kinesis_client.put_records(
              StreamName='bigcrime-streaming', Records=for_kinesis
      )
      count += len(for_kinesis)
      ts_2 = time.time()
      diff = ts_2 - ts_1
      if diff > 5.0:
        print(f'rate = {count / diff}/sec')
        ts_1 = time.time()
        count = 0
      # time.sleep(random.randint(10, 100)/1000.0)
      # print(f'{for_kinesis}')
    if size < limit:
      break
    offset += limit
  date_from = date_from + timedelta(days=8)
  offset = 0

requesting data from 2001-01-01 to 2001-01-08
rate = 861.1113560170162/sec
requesting data from 2001-01-09 to 2001-01-16
rate = 939.4156790262856/sec
rate = 1119.1058462268213/sec
requesting data from 2001-01-17 to 2001-01-24
rate = 940.2327345662626/sec
rate = 1141.9273691951641/sec
requesting data from 2001-01-25 to 2001-02-01
rate = 936.8075536510506/sec
requesting data from 2001-02-02 to 2001-02-09
rate = 895.0305275868236/sec
rate = 1138.0613238755157/sec
requesting data from 2001-02-10 to 2001-02-17
rate = 923.2972392894261/sec
rate = 1152.6914599004642/sec
requesting data from 2001-02-18 to 2001-02-25
rate = 935.1995587701152/sec
requesting data from 2001-02-26 to 2001-03-05
rate = 934.3970828794897/sec
rate = 1138.4343996483117/sec
requesting data from 2001-03-06 to 2001-03-13
rate = 920.491576190238/sec
rate = 1145.4305622397762/sec
requesting data from 2001-03-14 to 2001-03-21
rate = 935.4902006133623/sec
requesting data from 2001-03-22 to 2001-03-29
rate = 968.9540031103618/

KeyboardInterrupt: 